In [46]:
# Import necessary modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

# Import dataset
cc_apps = pd.read_csv("crx.data", header = None)
cc_apps.head(10)
cc_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    object 
 13  13      690 non-null    object 
 14  14      690 non-null    int64  
 15  15      690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


In [47]:
# Describe, drop non-essential columns, and split dataset
cc_apps.describe()
cc_apps = cc_apps.drop(columns = [11,13])
cc_apps_train, cc_apps_test = train_test_split(cc_apps, test_size=0.33, random_state=42)

In [48]:
# Replace ? with NaN
cc_apps_train_nans_replaced = cc_apps_train.replace("?", np.NaN)
cc_apps_test_nans_replaced = cc_apps_test.replace("?",np.NaN)

# Checking for object data type. If object, impute most frequent values, else impute mean value
for col in cc_apps_train_nans_replaced:
    if cc_apps_train_nans_replaced[col].dtypes == "object":
        cc_apps_train_imputed = cc_apps_train_nans_replaced.fillna(cc_apps_train_nans_replaced[col].value_counts().index[0])
        cc_apps_test_imputed = cc_apps_test_nans_replaced.fillna(cc_apps_train_nans_replaced[col].value_counts().index[0])
    else:
        cc_apps_train_imputed = cc_apps_train_nans_replaced.fillna(cc_apps_train_nans_replaced[col].mean())
        cc_apps_test_imputed = cc_apps_test_nans_replaced.fillna(cc_apps_train_nans_replaced[col].mean())

print(cc_apps_train_imputed.isnull().sum())
print(cc_apps_test_imputed.isnull().sum())

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64


In [49]:
# Converting categorical values
cc_apps_train_cat_encoding = pd.get_dummies(cc_apps_train_imputed)
cc_apps_test_cat_encoding = pd.get_dummies(cc_apps_test_imputed)

# Reindex test dataset
cc_apps_test_cat_encoding = cc_apps_test_cat_encoding.reindex(columns = cc_apps_train_cat_encoding.columns, fill_value=0)

In [50]:
# Segregate data into X and y
X_train, y_train = cc_apps_train_cat_encoding.iloc[:, :-1].values, cc_apps_train_cat_encoding.iloc[:,[-1]].values
X_test, y_test = cc_apps_test_cat_encoding.iloc[:,:-1].values, cc_apps_test_cat_encoding.iloc[:,[-1]].values

# Instantiate and fit transform MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

In [51]:
# Instantiate LogisticRegression
logreg = LogisticRegression()
# Fit logreg into train dataset
logreg.fit(rescaledX_train, y_train)
y_pred = logreg.predict(rescaledX_test)
# Print confusion matrix
print(confusion_matrix(y_test, y_pred))

[[103   0]
 [  0 125]]


c:\Users\ahron\ahron.anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [52]:
# Creating tol and max_iter lists
tol = [0.1,0.001,0.0001]
max_iter = [100,150,200]
# Creating a param_grid dict
param_grid = dict(tol=tol, max_iter = max_iter)
# Instantiate and fit GridSearchCV
grid_model = GridSearchCV(logreg, param_grid, cv=5)
grid_model_result = grid_model.fit(rescaledX_train, y_train)
# Evaluate best parmaeters and best score
best_params, best_score = grid_model_result.best_params_, grid_model_result.best_score_
print("Best: %f using %s" %(best_score, best_params))
# Best model
best_model = grid_model_result.best_estimator_
print("Accuracy of logistic regression classfier: ", best_model.score(rescaledX_test, y_test))

c:\Users\ahron\ahron.anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ahron\ahron.anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ahron\ahron.anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ahron\ahron.anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when 

Best: 1.000000 using {'max_iter': 100, 'tol': 0.001}
Accuracy of logistic regression classfier:  1.0


c:\Users\ahron\ahron.anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ahron\ahron.anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ahron\ahron.anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ahron\ahron.anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when 